In [1]:
# %%
# %%
# -*- coding: utf-8 -*-
"""
Created on Fri Mar  4 13:59:24 2022

@author: zapatacasv
"""

import pandas as pd
import numpy as np
import os
import xarray as xr 
import plotly.express as px

os.getcwd()
DIR = "X:\\user\\zapatacasv\\sdg7\\scripts"   # Set the running directory here
os.chdir(DIR)



In [50]:
a = xr.open_dataset('..\\output\\PEP-2C-AdvPE-DLS\\ec_a_world.nc') #The capacities required per tech only vary for DLS or not DLS scenarios
a = a.sel(year = 2030)



In [51]:
#Calculating capacity for Stand-alone because I forgot to export it
import data_import as dat

import test as gph
gph.Elecuseg = xr.open_dataarray('..\\output\\gph_SSP2_DLS\\gph_Elecuseg_world_SSP2.nc').sel(year = 2030)
gph.hhnumberg = xr.open_dataset('..\\output\\gph_SSP2_DLS\\gph_hhnumberg_world_SSP2.nc').sel(year = 2030)
gph.popg = xr.open_dataset('..\\output\\gph_SSP2_DLS\\gph_popg_world_SSP2.nc').sel(year = 2030)

# !******************** PV STANDALONE ***********************

MG2SA	=	1.2			#! Multiplication factor from mini-gridSAl_SystemCapacity0 to standalone price


elec_use_perhh = dat.elec_use_perhh.sel( turq = 1, year = 2030)
a['SAl_SystemCapacity0'] = xr.full_like(gph.Elecuseg, np.nan)
for r in dat.nrc:
        a['SAl_SystemCapacity0'] = xr.where((a.imager == r) ,elec_use_perhh.sel(region = r), a['SAl_SystemCapacity0'])   # (kWh-use per hh) 


a['SAl_SystemCapacity'] = a['SAl_SystemCapacity0'].expand_dims({'dos':['PV', 'diesel']}).copy().transpose('y', 'x',  'dos')  #		



##!************************* STANDALONE DIESEL GENSET ***********************
DG_HoursOperation = 24*365
DGStAl_LoadFactor		=	0.5 

a['SAl_SystemCapacity'][dict(dos= 1)] = xr.where((gph.hhnumberg.hhnumberg.sel(turq = 1) > 1e-6), a['SAl_SystemCapacity0']*1.7/(DG_HoursOperation*DGStAl_LoadFactor), 0 ). transpose('y', 'x')  # (kW) capcity per TURQ total ---- The 1.7 is the peak versus middle load ratio from my paper

In [47]:
#list of scenarios
scen = ['SSP2','SSP2-DLS',  'PEP-2C-AdvPE', 'PEP-2C-AdvPE-DLS']
fn = ['techid_univ', 'techid_bl']
path = []
fn2 = []
for s in scen:
    for f in fn:
        arr = '..\\output\\%s\\%s.nc' %(s, f)
        path.append(arr)
        arr = f + '-' + s #s #
        fn2.append(arr)
techid1 = xr.combine_by_coords([xr.open_dataarray(path[i]).to_dataset(name = fn2[i] ) for i in range(len(path))], combine_attrs = "override")



In [52]:

fn2 = [#'techid_univ-SSP2',
 #'techid_bl-SSP2',
 'techid_univ-SSP2-DLS',
 #'techid_univ-PEP-2C-AdvPE']#,
 'techid_univ-PEP-2C-AdvPE-DLS']

scen = {'techid_univ-SSP2' : 'SSP2-UA',
 'techid_bl-SSP2' :  'SSP2-BL',
 'techid_univ-SSP2-DLS' :  'SSP2-UA-DLS',
 'techid_univ-PEP-2C-AdvPE' :  'SSP2-2C-UA', #} #,
 #'techid_bl-PEP-2C-AdvPE' : 'SSP2-2C-BL',
 'techid_univ-PEP-2C-AdvPE-DLS' : 'SSP2-2C-UA-DLS'}

techid = techid1[fn2]

In [53]:

capacity = techid

for f in fn2:
    capacity[scen[f]] = techid[f]
    #capacity =  xr.where(techid == 1, np.nan, capacity)   #! Grid cell already has access to central grid
    #capacity = xr.where(techid == 2, np.nan, capacity) #! Distribution network plus generation cost  
    capacity[scen[f]] = xr.where(techid[f] <= 3, np.nan, capacity[scen[f]])    #! Transmission and Distribution network plus generation cost  
    capacity[scen[f]] = xr.where(techid[f] == 4, a.MiniG_ActualSyscap.sel(seis ='PV' ), capacity[scen[f]]) #Mini grid
    capacity[scen[f]] = xr.where(techid[f] == 5, a.MiniG_ActualSyscap.sel(seis = 'diesel'), capacity[scen[f]])
    capacity[scen[f]] = xr.where(techid[f] == 6, a.MiniG_ActualSyscap.sel(seis = 'wind'), capacity[scen[f]])
    capacity[scen[f]] = xr.where(techid[f] == 7, a.MiniG_ActualSyscap.sel(seis = 'hydro'), capacity[scen[f]])
    capacity[scen[f]] = xr.where(techid[f] == 8, a['PVHybSystemCapacity']+a[ 'DGHybSystemCapacity'], capacity[scen[f]]) # #Mini grid hybrid PV
    capacity[scen[f]] = xr.where(techid[f] == 9, a['WindHybSystemCapacity']+a[ 'DGHybSystemCapacity2'], capacity[scen[f]])
    capacity[scen[f]] = xr.where(techid[f] == 10,a['SAl_SystemCapacity'].sel( dos = 'PV'), capacity[scen[f]])     # Standalone turq = 1,
    capacity[scen[f]] = xr.where(techid[f] == 11,a['SAl_SystemCapacity'].sel( dos = 'diesel'), capacity[scen[f]]) #turq = 1,
    capacity[f] = techid[f]


capacity.to_netcdf('..\\output\\analysis\\ec_system_capacities_SSP2_DLS.nc')

In [54]:
#Preparing data for Plotting distribution of least cost options in the population for achieving universal access for WORLD for offgrid
arr = capacity
#nrc = techid.imager.where(techid['techid_univ-SSP2-DLS']>0, drop = True).groupby('imager')
df1 = pd.DataFrame([])
for r in dat.nrc-1: #[3,4,6, 8.0, 9.0,10,21,22,24,25,26, 18, 3]:
    for f in fn2:
        df = arr.where(arr.imager == r)
        df.coords[f] = df[f]
        if np.isnan(np.min(df[f])):
            pass
        else:
            df2 = df[scen[f]].groupby(df[f]).sum().drop(['spatial_ref',  'dos', 'year', 'seis', 'tres', 'turq']).to_dataframe(name = ('capacity_kW'))
            df2['region'] = r
            df2['scenario'] = scen[f]
            df1 = pd.concat([df1,df2])#df1.append(df)
#df1

df2 = df1
df2 = df2.reset_index(names = 'techid')
df2 = df2.drop(df2[df2.techid<4].index)
df2 = df2[df2["scenario"].isin(scen.values())]
df2 = df2.replace({'techid': {  4: 'MG-PV', 5: 'MG-diesel', 6: 'MG-wind', 7:'MG-hydro' , 8:'MG-PV-diesel', 9:'MG-wind-diesel', 10:'SA-PV', 11: "SA-diesel"}}) #1: 'already access',
#df2 = df2.replace({'region':  {4: 'Rest Central America', 6:'Rest South America', 8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa', 21: 'Southeastern Asia', 22: 'Indonesia +', 24:'Oceania' , 25:'Rest South Asia', 26:'Rest South Africa', 18: 'India', 3: 'Mexico'}})  #{ 8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa', 26:'Rest SSA'}}) #{6:'Rest South America', 8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa', 21: 'Southeastern Asia', 22: 'Indonesia +', 24:'Oceania' , 25:'Rest S.Asia', 26:'Rest S.Africa'}})
df2['capacity_GW'] = df2['capacity_kW']*1e-6
df2.to_csv('..\\output\\analysis\\capacity_pertech_scenario_world_SSP2_DLS.csv')

In [55]:
#Asessment for central gid connection
Elecuseg = gph.Elecuseg# xr.open_dataarray('..\\output\\SSP2-DLS\\gph_Elecuseg_world_SSP2.nc')

arr = techid
arr = xr.where((arr ==2) | (arr==3), arr, np.nan)
arr['Elecuseg'] = Elecuseg#.sel(year = 2030)  

cg = pd.DataFrame([])
for f in fn2:
        df = arr[[f, 'Elecuseg']]
        df = df.Elecuseg.where(arr[f]>0).groupby('imager').sum().drop(['spatial_ref',  'dos', 'year', 'seis', 'tres', 'turq', 'region']).to_dataframe(name = ('Elecuseg'))
        df['scenario'] = scen[f]
        cg = pd.concat([cg,df])#df1.append(df)

cg2 = cg
cg2 = cg2.reset_index(names = 'region')
#df2 = df2.drop(df2[df2.techid<4].index)
cg2 = cg2[cg2["scenario"].isin(scen.values())]
#cg2 = cg2.replace({'techid': { 2:'CG-densification', 3: 'CG-extension',}}) #1: 'already access',
#cg2 = cg2.replace({'region':  {4: 'Rest Central America', 6:'Rest South America', 8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa', 21: 'Southeastern Asia', 22: 'Indonesia +', 24:'Oceania' , 25:'Rest South Asia', 26:'Rest South Africa', 18: 'India', 3: 'Mexico'}})  #{ 8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa', 26:'Rest SSA'}}) #{6:'Rest South America', 8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa', 21: 'Southeastern Asia', 22: 'Indonesia +', 24:'Oceania' , 25:'Rest S.Asia', 26:'Rest S.Africa'}})

cg2.to_csv('..\\output\\analysis\\elec_use_CGextension_scenario_DLS.csv')

Concatenate in excel dls and non dls for cg and off-grid. Also calculate from cg elecuse the capacities using regional capacity factors

In [ ]:
#preparing data for global map array([ 3,  4,  6,  7,  8,  9, 10, 17, 18, 19, 20, 21, 22, 24, 25, 26])
df2 = pd.read_csv('..\\output\\analysis\\capacity_pertech_scenario_world.csv')  #concatanated in excel the dls and non dls scenarios
df2 = df2.replace({'region':  {3: 'Mexico', 4: 'Rest Central America', 6:'Rest South America',  7:'Northern Africa',8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa',17:'Middle East', 18:'India +', 19: 'Korea', 20:'China +', 21: 'Southeastern Asia', 22: 'Indonesia +', 24:'Oceania' , 25:'Rest South Asia', 26:'Rest South Africa'}}) 
# df2 = df2.replace({'region':  {3: 'Rest World', 4: 'Rest World', 6:'Rest World',  7:'Rest World',8:'Western Africa', 9: 'Eastern Africa',
#                                 10: 'Rest World',17:'Rest World', 18:'Rest World', 19: 'Rest World', 20:'Rest World', 21: 'Rest World',
#                                   22:'Rest World', 24:'Rest World', 25:'Rest South Asia', 26:'Rest South Africa'}}) 
df2 = df2.groupby(['region', 'scenario']).sum()
df2 = df2.reset_index()
df2['techid'] = 'Off-Grid'
df2 = df2.sort_values(by='capacity_MW', ascending=False)

df1 = pd.read_csv('..\\output\\analysis\\capacity_CGextension_allscenarios.csv')  #concatanated in excel the dls and non dls scenarios and converted to capacity 
df1 = df1[df1['region'].isin([ 3,  4,  6,  7,  8,  9, 10, 17, 18, 19, 20, 21, 22, 24, 25, 26])]
#df1 = df1.replace({'region':  {3: 'Mexico', 4: 'Rest Central America', 6:'Rest South America',  7:'Northern Africa',8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa',17:'Middle East', 18:'India +', 19: 'Korea', 20:'China +', 21: 'Southeastern Asia', 22: 'Indonesia +', 24:'Oceania' , 25:'Rest South Asia', 26:'Rest South Africa'}}) 
df1 = df1.replace({'region':  {3: 'Rest World', 4: 'Rest World', 6:'Rest World',  7:'Rest World',8:'Western Africa', 9: 'Eastern Africa', 10: 'Rest World',17:'Rest World', 18:'Rest World', 19: 'Rest World', 20:'Rest World', 21: 'Rest World', 22:'Rest World', 24:'Rest World', 25:'Rest South Asia', 26:'Rest South Africa'}}) 
df1 = df1.reset_index()
df1['techid'] = 'CG Extension T&D'
df1 =df1.drop(columns = 'Elecuseg')
df1 = df1.sort_values(by='capacity_MW', ascending=False)

df3 = pd.concat([df1,df2])
df3['capacity_GW'] = df3['capacity_MW']/1000